In [ ]:
import torch
import torch.nn as nn
import tiktoken
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import re


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": True        # Query-Key-Value bias
}

In [ ]:
torch.manual_seed(123)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec


In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [ ]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"], 
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

In [ ]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [ ]:
#tokenizer = tiktoken.get_encoding("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:

batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"
txt3 = "Maybe I can play"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch.append(torch.tensor(tokenizer.encode(txt3)))
batch = torch.stack(batch, dim=0)
print(batch)

In [ ]:
model = GPTModel(GPT_CONFIG_124M)

In [ ]:
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

In [ ]:
#torch.nn.functional.softmax(out[0][0]).argmax()
out[0][3].argmax()


In [ ]:
tokenizer.decode(batch[0].tolist() + [28931])

In [ ]:
tokenizer.convert_ids_to_tokens(28931)

In [ ]:
gpt2_hf=GPT2LMHeadModel.from_pretrained('gpt2')


In [ ]:
from_to = {'transformer.wte.weight': 'tok_emb.weight',
           'transformer.wpe.weight': 'pos_emb.weight',
           'transformer.h.#.ln_1.weight': 'trf_blocks.{num}.norm1.scale',
           'transformer.h.#.ln_1.bias': 'trf_blocks.{num}.norm1.shift',
           'transformer.h.#.ln_2.weight': 'trf_blocks.{num}.norm2.scale',
           'transformer.h.#.ln_2.bias': 'trf_blocks.{num}.norm2.shift',
           'transformer.h.#.attn.c_attn.weight': 'trf_blocks.{num}.att.W_query.weight,trf_blocks.{num}.att.W_key.weight,trf_blocks.{num}.att.W_value.weight',
           'transformer.h.#.attn.c_attn.bias': 'trf_blocks.{num}.att.W_query.bias,trf_blocks.{num}.att.W_key.bias,trf_blocks.{num}.att.W_value.bias',
           'transformer.h.#.attn.c_proj.weight': 'trf_blocks.{num}.att.out_proj.weight',
           'transformer.h.#.attn.c_proj.bias': 'trf_blocks.{num}.att.out_proj.bias',
           'transformer.h.#.mlp.c_proj.weight': 'trf_blocks.{num}.ff.layers.0.weight',
           'transformer.h.#.mlp.c_fc.bias': 'trf_blocks.{num}.ff.layers.0.bias',
           'transformer.h.#.mlp.c_fc.weight': 'trf_blocks.{num}.ff.layers.2.weight',
           'transformer.h.#.mlp.c_proj.bias': 'trf_blocks.{num}.ff.layers.2.bias'
}

In [ ]:
state=gpt2_hf.state_dict()
state_model=model.state_dict()
for k,v in state.items():
    if k == 'transformer.wte.weight' or k == 'transformer.wpe.weight':
        state_model[from_to[k]] = state[k]
    else:
        k_parts = k.split('.')
        layer = -1
        if len(k_parts) >= 3 and k_parts[2] and re.search(pattern=r'^[0-9]*$', string=k_parts[2]):
            layer = k_parts[2]
            k_parts[2] = '#'
            key = '.'.join(k_parts)
            if key == 'transformer.h.#.attn.c_attn.weight' or key == 'transformer.h.#.attn.c_attn.bias':
                if key == 'transformer.h.#.attn.c_attn.weight':
                    wq = state[k][:,:768]
                    wk = state[k][:,768:1536]
                    wv = state[k][:,1536:]
                else:
                    wq = state[k][:768]
                    wk = state[k][768:1536]
                    wv = state[k][1536:]
                q, k, v = from_to[key].format(num=layer).split(',')
                state_model[q] = wq
                state_model[k] = wk
                state_model[v] = wv
            elif from_to[key] != None:
                aux = from_to[key].format(num=layer)
                if state_model[aux].shape != state[k].shape:
                    print(f"state {k} shape {state[k].shape} doesn't match for {aux} shape {state_model[aux].shape}")
                state_model[aux] = state[k]
                

In [ ]:
model_l_gpt = GPTModel(GPT_CONFIG_124M)
model_l_gpt.load_state_dict(state_dict=state_model)

In [ ]:
out = model_l_gpt(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)


In [ ]:
next = torch.nn.functional.softmax(out[0][3], dim=0).argmax()
next

In [ ]:
tokenizer.decode(batch[0].tolist() + [next])

In [ ]:
next = torch.nn.functional.softmax(out[1][3], dim=0).argmax()
next

In [ ]:
tokenizer.decode(batch[1].tolist() + [next])

In [ ]:
next = torch.nn.functional.softmax(out[2][3], dim=0).argmax()
next

In [ ]:
tokenizer.decode(batch[2].tolist() + [next])